In [2]:
# Install necessary libraries
!pip install rdflib tensorflow scikit-learn pandas numpy

# Import necessary libraries
from rdflib import Graph
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input
from sklearn.metrics import mean_squared_error
import numpy as np
from google.colab import files



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 12.3 MB/s eta 0:00:00


In [3]:
import rdflib
from rdflib import URIRef, Literal, Graph, RDF, Namespace, plugin
from rdflib.plugins import sparql
from rdflib.plugins.sparql import prepareQuery
from rdflib.parser import Parser
import numpy as np
import networkx as nx
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from rdflib import Graph, URIRef, BNode, Literal
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

In [6]:
from google.colab import files
from rdflib import Graph
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Prompt the user to upload a file
uploaded = files.upload()

# Parse the uploaded RDF file
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

    # Create an RDF graph
    g = Graph()

    # Attempt to parse the RDF file
    try:
        # Adjust the format as necessary (e.g., 'xml', 'turtle', 'ntriples', 'json-ld')
        g.parse(filename, format='turtle')
        print(f"Successfully parsed RDF file: {filename}")
    except Exception as e:
        print(f"Error parsing RDF file: {e}")

# Define SPARQL query
query = prepareQuery("""
   SELECT * {
      ?s rdf:type <https://w3id.org/def/foo#gPSObservation>;
         <https://w3id.org/def/foo#localDate> ?LocalDate;
         <https://w3id.org/def/foo#localTime> ?localTime;
         <http://www.w3.org/2003/01/geo/wgs84_pos#latitude> ?lat;
         <http://www.w3.org/2003/01/geo/wgs84_pos#longitude> ?long .
   }
 """)

# Extract features and labels
features, labels = [], []
for row in g.query(query):
    try:
        long = float(row.long.toPython())
        lat = float(row.lat.toPython())
        local_date = str(row.LocalDate.toPython())
        local_time = str(row.localTime.toPython())

        # Convert date and time to numerical values (e.g., lengths)
        local_date_num = len(local_date)  # Example: Length of the date string
        local_time_num = len(local_time)  # Example: Length of the time string

        features.append([long, lat, local_date_num, local_time_num])
        labels.append([long, lat, local_date_num, local_time_num])
    except Exception as e:
        print(f"Skipping invalid row: {e}")

# Validate data before conversion
if not features or not labels:
    print("Error: No valid data extracted. Check your RDF query or data.")
    exit()

# Convert to NumPy arrays
try:
    features_np = np.array(features, dtype=np.float32)
    labels_np = np.array(labels, dtype=np.float32)
except ValueError as e:
    print(f"Error converting features or labels to NumPy arrays: {e}")
    exit()

# Split data
X_train_temp, X_test, y_train_temp, y_test = train_test_split(features_np, labels_np, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=42)

# Train model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),  # Features have 4 dimensions
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='linear')  # Predicting 4 features
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss}, Test Mean Absolute Error: {test_mae}")

# Predict on test data
y_pred = model.predict(X_test)

# Calculate RMSE for all features
rmse_long = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))  # Longitude
rmse_lat = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))  # Latitude
rmse_date = np.sqrt(mean_squared_error(y_test[:, 2], y_pred[:, 2]))  # localDate
rmse_time = np.sqrt(mean_squared_error(y_test[:, 3], y_pred[:, 3]))  # localTime

print(f"Test Longitude RMSE: {rmse_long}")
print(f"Test Latitude RMSE: {rmse_lat}")
print(f"Test localDate RMSE: {rmse_date}")
print(f"Test localTime RMSE: {rmse_time}")


Saving Seri(3).ttl to Seri(3) (2).ttl
User uploaded file "Seri(3) (2).ttl" with length 10482711 bytes
Successfully parsed RDF file: Seri(3) (2).ttl
Epoch 1/500
172/172 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 804.2060 - mean_absolute_error: 11.4421 - val_loss: 0.0029 - val_mean_absolute_error: 0.0316
Epoch 2/500
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0029 - mean_absolute_error: 0.0293 - val_loss: 0.0027 - val_mean_absolute_error: 0.0285
Epoch 3/500
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0029 - mean_absolute_error: 0.0290 - val_loss: 0.0028 - val_mean_absolute_error: 0.0287
Epoch 4/500
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0030 - mean_absolute_error: 0.0291 - val_loss: 0.0043 - val_mean_absolute_error: 0.0319
Epoch 5/500
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0034 - mean_absolute_error: 0.0302 - val_loss: 0.0036 - val_mean_absolute_error: 0.0304
Epoch 6/500
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0032 - mean_absolute_error: 0.0

In [10]:
# Simulate predictions and ground truth data (as placeholders)
np.random.seed(42)  # For reproducibility

# Assume 100 test samples
num_samples = 100
y_test_simulated = np.random.rand(num_samples, 4)  # Ground truth
y_pred_simulated = y_test_simulated + np.random.normal(0, 0.05, y_test_simulated.shape)  # Predictions with slight noise

# Prepare predictions and ground truth for exporting
predictions = pd.DataFrame({
    'Longitude_Predicted': y_pred_simulated[:, 0],
    'Latitude_Predicted': y_pred_simulated[:, 1],
    'localDate_Predicted': y_pred_simulated[:, 2],
    'localTime_Predicted': y_pred_simulated[:, 3],
    'Longitude_Actual': y_test_simulated[:, 0],
    'Latitude_Actual': y_test_simulated[:, 1],
    'localDate_Actual': y_test_simulated[:, 2],
    'localTime_Actual': y_test_simulated[:, 3]
})

# Save predictions to CSV
output_file = 'Seri.csv'
predictions.to_csv(output_file, index=False)

output_file


'Seri.csv'

In [ ]:
# GPS collar
import csv
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD
from datetime import datetime

# Define namespaces
FOO = Namespace("https://w3id.org/def/foo#")
SOSA = Namespace("http://www.w3.org/ns/sosa/")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")

# Create a new graph
g = Graph()
# Bind namespaces
g.bind("foo", FOO)
g.bind("sosa", SOSA)
g.bind("geo", GEO)

# Path to your CSV file
csv_file_path = "Seri.csv"  # Adjust the path accordingly

# Read and process data
with open(csv_file_path, newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')

    # Print headers for debugging
    print("Headers:", reader.fieldnames)

    for row in reader:
        identifier = row.get('ID', '').strip()
         # local_date = row.get('LocalDate', '').strip()
        local_date = datetime.strptime(row.get('LocalDate', '').strip(), '%Y-%m-%d').isoformat() if row.get('LocalDate', '').strip() and '00' not in row.get('LocalDate', '') else None
        # local_time = row.get('LocalTime', '').strip()
        local_time = (datetime.strptime(row.get('LocalTime', '').strip(), '%H:%M:%S').time().isoformat() if ':' in row.get('LocalTime', '').strip() and 'AM' not in row.get('LocalTime', '').strip() and 'PM' not in row.get('LocalTime', '').strip() else datetime.strptime(row.get('LocalTime', '').strip().replace('AM', ' AM').replace('PM', ' PM'), '%I:%M:%S %p').time().isoformat()) if row.get('LocalTime', '').strip() else None
        # gmt_date = row.get('GMTDate', '').strip()
        gmt_date = (datetime.strptime(row.get('GMTDate', '').strip(), '%Y-%m-%d').isoformat() if '-' in row.get('GMTDate', '').strip() else datetime.strptime(row.get('GMTDate', '').strip(), '%m/%d/%Y').isoformat()) if row.get('GMTDate', '').strip() and '00' not in row.get('GMTDate', '') else None
        # gmt_time = row.get('GMTTime', '').strip()
        gmt_time = (datetime.strptime(row.get('GMTTime', '').strip(), '%H:%M:%S').time().isoformat() if ':' in row.get('GMTTime', '').strip() and 'AM' not in row.get('GMTTime', '').strip() and 'PM' not in row.get('GMTTime', '').strip() else datetime.strptime(row.get('GMTTime', '').strip().replace('AM', ' AM').replace('PM', ' PM'), '%I:%M:%S %p').time().isoformat()) if row.get('GMTTime', '').strip() else None
        lat = row.get('lat', '').strip()
        long = row.get('long', '').strip()
        temperature = row.get('Temperature', '').strip()
        extTemp = row.get('ExtTemp', '').strip()
        activity = row.get('Activity', '').strip()
        speed = row.get('Speed', '').strip()
        direction = row.get('Direction', '').strip()
        altitude = row.get('Altitude', '').strip()
        cov = row.get('Cov', '').strip()
        hdop = row.get('HDOP', '').strip()
        distance = row.get('Distance', '').strip()
        count = row.get('Count', '').strip()
        # Create observation URI
        observation = URIRef(f"https://w3id.org/def/foo#{identifier}")
        g.add((observation, RDF.type, FOO.gPSObservation))
        g.add((observation, FOO.id, Literal(identifier, datatype=XSD.string)))
        g.add((observation, FOO.madeBySensor, FOO.seriGPS))  # Link sensor to observation
        g.add((observation, FOO.hasFeatureOfInterest, FOO.seri))
        # Add observation properties
        g.add((observation, FOO.localDate, Literal(local_date, datatype=XSD.date)))
        g.add((observation, FOO.localTime, Literal(local_time, datatype=XSD.time)))
        g.add((observation, GEO.latitude, Literal(lat, datatype=XSD.double)))
        g.add((observation, GEO.longitude, Literal(long, datatype=XSD.double)))


# Serialize the graph to a file
output_file = "Seri_rdf_graph.ttl"
g.serialize(destination=output_file, format="turtle")

print(f"Knowledge graph has been serialized to {output_file}")



In [ ]:
# Send the RDF data to Stardog
stardog_url = "http://localhost:5820/your-database-name"
stardog_user = "admin"
stardog_password = "admin"

headers = {
    "Content-Type": "text/turtle",
    "Accept": "application/sparql-results+json"
}

response = requests.post(
    f"{stardog_url}/update",
    data=rdf_data,
    auth=(stardog_user, stardog_password),
    headers=headers
)

# Check response
if response.status_code == 200:
    print("Data successfully uploaded to Stardog!")
else:
    print(f"Error uploading data: {response.status_code}, {response.text}")